In [1]:
import os
import random
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import KFold

from kerashypetune import KerasGridSearch, KerasGridSearchCV

In [2]:
def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

np.random.seed(33)
subset_train = np.random.uniform(0,1, len(y_train))
subset_test = np.random.uniform(0,1, len(y_test))

x_train = x_train[subset_train < 0.1] / 255
y_train = y_train[subset_train < 0.1]

x_test = x_test[subset_test < 0.3] / 255
y_test = y_test[subset_test < 0.3]

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((6036, 28, 28), (6036,), (2963, 28, 28), (2963,))

In [4]:
def get_model(param):
    
    set_seed(33)
            
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(param['unit_1'], activation='relu'))
    model.add(Dense(param['unit_2'], activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=param['lr']), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [5]:
param_grid = {
    'unit_1': [128,64], 
    'unit_2': [64,32],
    'lr': [1e-2,1e-3], 
    'epochs': 100, 
    'batch_size': 512
}

# Fixed validation search

In [6]:
es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)
# callable(hypermodel) ==> True

kgs = KerasGridSearch(hypermodel, param_grid, monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kgs.search(x_train, y_train, validation_data=(x_test, y_test), callbacks=[es])


8 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/8) *****
Search({'unit_1': 128, 'unit_2': 64, 'lr': 0.01, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
SCORE: 0.95208 at epoch 17

***** (2/8) *****
Search({'unit_1': 128, 'unit_2': 64, 'lr': 0.001, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00038: early stopping
SCORE: 0.95005 at epoch 28

***** (3/8) *****
Search({'unit_1': 128, 'unit_2': 32, 'lr': 0.01, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
SCORE: 0.9514 at epoch 14

***** (4/8) *****
Search({'unit_1': 128, 'unit_2': 32, 'lr': 0.001, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00053: early stopping
SCORE: 0.95039 at epoch 47

***** (5/8) *****
Search({'unit_1': 64, 'unit_2': 64, 'lr': 0.01, 'e

In [7]:
kgs.scores

[0.95208, 0.95005, 0.9514, 0.95039, 0.95343, 0.94398, 0.9487, 0.93993]

In [8]:
kgs.best_score

0.95343

In [9]:
kgs.best_params

{'unit_1': 64,
 'unit_2': 64,
 'lr': 0.01,
 'epochs': 41,
 'batch_size': 512,
 'steps_per_epoch': 12}

In [10]:
kgs.best_model

# Fixed validation search Generator

In [11]:
gen = ImageDataGenerator(
         rotation_range=90,
         width_shift_range=0.1,
         height_shift_range=0.1,
         zoom_range=0.2)

In [12]:
es = EarlyStopping(patience=5, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)
# callable(hypermodel) ==> True

kgs = KerasGridSearch(hypermodel, param_grid, monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kgs.search(gen.flow(np.expand_dims(x_train,-1), y_train, batch_size=param_grid['batch_size'], seed=33), 
           validation_data=(np.expand_dims(x_test,-1), y_test), 
           callbacks=[es], steps_per_epoch=len(x_train)//param_grid['batch_size'])


8 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/8) *****
Search({'unit_1': 128, 'unit_2': 64, 'lr': 0.01, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
SCORE: 0.85353 at epoch 15

***** (2/8) *****
Search({'unit_1': 128, 'unit_2': 64, 'lr': 0.001, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00031: early stopping
SCORE: 0.86905 at epoch 26

***** (3/8) *****
Search({'unit_1': 128, 'unit_2': 32, 'lr': 0.01, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00049: early stopping
SCORE: 0.90348 at epoch 44

***** (4/8) *****
Search({'unit_1': 128, 'unit_2': 32, 'lr': 0.001, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00035: early stopping
SCORE: 0.86365 at epoch 34

***** (5/8) *****
Search({'unit_1': 64, 'unit_2': 64, 'lr': 0.01, '

In [13]:
kgs.scores

[0.85353, 0.86905, 0.90348, 0.86365, 0.86466, 0.82889, 0.83733, 0.8353]

In [14]:
kgs.best_score

0.90348

In [15]:
kgs.best_params

{'unit_1': 128,
 'unit_2': 32,
 'lr': 0.01,
 'epochs': 44,
 'batch_size': 512,
 'steps_per_epoch': 11}

In [16]:
kgs.best_model

# Cross validation search

In [17]:
X = np.concatenate([x_train, x_test])
y = np.concatenate([y_train, y_test])

X.shape, y.shape

((8999, 28, 28), (8999,))

In [18]:
cv = KFold(n_splits=3, random_state=33, shuffle=True)

es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)

kgs = KerasGridSearchCV(hypermodel, param_grid, cv=cv, monitor='val_accuracy', greater_is_better=True)
kgs.search(X, y, callbacks=[es])


##################
###  Fold 001  ###
##################

8 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/8) *****
Search({'unit_1': 128, 'unit_2': 64, 'lr': 0.01, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
SCORE: 0.94767 at epoch 22

***** (2/8) *****
Search({'unit_1': 128, 'unit_2': 64, 'lr': 0.001, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00056: early stopping
SCORE: 0.944 at epoch 56

***** (3/8) *****
Search({'unit_1': 128, 'unit_2': 32, 'lr': 0.01, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00026: early stopping
SCORE: 0.94567 at epoch 23

***** (4/8) *****
Search({'unit_1': 128, 'unit_2': 32, 'lr': 0.001, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
SCORE: 0.935 at epoch 19

***** (5/8) *

In [19]:
kgs.folds_scores

{'fold 1': [0.94767, 0.944, 0.94567, 0.935, 0.94667, 0.93433, 0.944, 0.93467],
 'fold 2': [0.958, 0.95267, 0.95633, 0.94733, 0.95233, 0.93967, 0.94733, 0.94],
 'fold 3': [0.95365,
  0.94698,
  0.95065,
  0.94532,
  0.94765,
  0.94265,
  0.94898,
  0.94365]}

In [20]:
kgs.folds_best_score

{'fold 1': 0.94767, 'fold 2': 0.958, 'fold 3': 0.95365}

In [21]:
kgs.folds_best_params

{'fold 1': {'unit_1': 128,
  'unit_2': 64,
  'lr': 0.01,
  'epochs': 22,
  'batch_size': 512,
  'steps_per_epoch': 12},
 'fold 2': {'unit_1': 128,
  'unit_2': 64,
  'lr': 0.01,
  'epochs': 23,
  'batch_size': 512,
  'steps_per_epoch': 12},
 'fold 3': {'unit_1': 128,
  'unit_2': 64,
  'lr': 0.01,
  'epochs': 19,
  'batch_size': 512,
  'steps_per_epoch': 12}}

In [22]:
kgs.folds_best_models

{'fold 1': <tensorflow.python.keras.engine.sequential.Sequential at 0x1500924ad48>,
 'fold 2': <tensorflow.python.keras.engine.sequential.Sequential at 0x15002b38408>,
 'fold 3': <tensorflow.python.keras.engine.sequential.Sequential at 0x15002b50f88>}

In [23]:
kgs.best_params_score

0.95311

In [24]:
kgs.best_params

[{'unit_1': 128,
  'unit_2': 64,
  'lr': 0.01,
  'epochs': 22,
  'batch_size': 512,
  'steps_per_epoch': 12},
 {'unit_1': 128,
  'unit_2': 64,
  'lr': 0.01,
  'epochs': 23,
  'batch_size': 512,
  'steps_per_epoch': 12},
 {'unit_1': 128,
  'unit_2': 64,
  'lr': 0.01,
  'epochs': 19,
  'batch_size': 512,
  'steps_per_epoch': 12}]